# Chapter 9: Building Complex Prompts

**Scenario:** You are building a specific tool, like a **Trip Planner** or a **Game Rule Assistant**. A simple one-line prompt isn't enough. You need to combine Rules, Context, Examples, and User Data into one big "Mega-Prompt".

In this notebook, you will learn:
- 🏗️ **The Architecture**: Breaking a big prompt into blocks (Context, Rules, Data).
- 🔧 **Variables**: Using Python to insert user data safely.
- 📜 **Reference Material**: Answering questions based on specific texts (like Game Rules).

---

## 1. Setup

Standard setup.

In [1]:
# Install tools
!pip install -q litellm python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')

In [3]:
def get_completion(prompt, system_prompt=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

## 2. The Anatomy of a Mega-Prompt

Complex prompts are easier if you build them in blocks. 
Think of it like LEGO bricks:

1.  **Role**: Who is the AI?
2.  **Context**: What is the goal?
3.  **Rules**: What are the limits?
4.  **Examples**: Show, don't just tell.
5.  **Data**: The user's input.

Let's build **"TravelBot"**.

In [4]:
# 1. Role & Context
ROLE = "You are TravelBot, an expert travel agent."
CONTEXT = "Your goal is to suggest 3 exciting activities based on the user's destination and interests."

# 2. Rules
RULES = """
- Be enthusiastic!
- Include estimated prices.
- Keep descriptions short (1 sentence).
"""

# 3. Examples (Few-Shot)
EXAMPLES = """
User: Paris, Art
TravelBot: 
1. Visit the Louvre ($17) - See the Mona Lisa!
2. Musee d'Orsay ($16) - Amazing Impressionist art.
3. Street Art in Belleville (Free) - Walk through an open-air gallery.
"""

# 4. Input Data (This varies per user)
user_input = "Tokyo, Food"

# Assemble the prompt
FINAL_PROMPT = f"""
{ROLE}
{CONTEXT}

Rules:
{RULES}

Examples:
{EXAMPLES}

Current Request: {user_input}
"""

print("--- Generated Prompt ---")
print(FINAL_PROMPT)
print("\n--- Model Response ---")
print(get_completion(FINAL_PROMPT, temperature=0.7, max_tokens=1024))

--- Generated Prompt ---

You are TravelBot, an expert travel agent.
Your goal is to suggest 3 exciting activities based on the user's destination and interests.

Rules:

- Be enthusiastic!
- Include estimated prices.
- Keep descriptions short (1 sentence).


Examples:

User: Paris, Art
TravelBot: 
1. Visit the Louvre ($17) - See the Mona Lisa!
2. Musee d'Orsay ($16) - Amazing Impressionist art.
3. Street Art in Belleville (Free) - Walk through an open-air gallery.


Current Request: Tokyo, Food


--- Model Response ---


1. Tsukiji Outer Market Food Tour ($100) - Sample fresh seafood and local delicacies!
2. Ramen Making Class ($80) - Learn to craft the perfect bowl of ramen!
3. Robot Restaurant Dinner Show ($90) - Enjoy a crazy, neon-lit dinner with robots and performers!



## 3. Handling Complex Rules (The Game Bot)

Sometimes you need the AI to follow a strict set of rules, like a rulebook for a board game.

Let's make a bot for a fictional game: **"Space Race"**.

In [5]:
GAME_RULES = """
1. Players start at Earth (Position 0).
2. Turn ordering: Roll a dice (1-6).
3. If you roll a 6, you move double the spaces (12 spaces).
4. If you roll a 1, you lose a turn (move 0 spaces).
5. Any other number, you move that many spaces.
"""

user_question = "I rolled a 6. What happens?"

PROMPT = f"""
You are the Referee for the game 'Space Race'. Answer based ONLY on the rules below.

<rules>
{GAME_RULES}
</rules>

Player Question: {user_question}
"""

print(get_completion(PROMPT, temperature=0.0, max_tokens=1024))

You move 12 spaces.



## 4. Exercises

### Exercise 1: The Code Helper

**Goal:** Build a "Code Helper" prompt. 
**Rule:** It must **Explain** the bug first, then **Fix** it.

Fill in the empty strings below.

In [6]:
BAD_CODE = "print('Hello World'"

ROLE = ""
RULES = ""
INPUT = f"Check this code:\n{BAD_CODE}"

PROMPT = f"""
{ROLE}

Rules:
{RULES}

{INPUT}
"""

# print(get_completion(PROMPT, temperature=0.0, max_tokens=1024))

## Summary

1.  **Block Structure**: Break big prompts into Role, Rules, Data.
2.  **Reference Data**: Use XML tags (`<rules>`) to hold rulebooks or documents.
3.  **Variables**: Use `f-strings` to swap in user data easily.